In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Collect all historical data

# For some reason i'm permabanned on FBREF english website lol
# So i scraped the /es/ website instead
def scrape_fbref_table_results(start_year, end_year, competition=''):
    all_tables = []
    headers = {
        'authority': 'forms.hscollectedforms.net',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'dnt': '1',
        'origin': 'https://fbref.com',
        'referer': 'https://fbref.com/en/comps/12/La-Liga-Stats',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }

    for i in range(int(start_year), int(end_year)):
        url = f'https://fbref.com/es/comps/12/{i}-{i + 1}/schedule/{i}-{i + 1}-La-Liga-Scores-and-Fixtures'
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        results_table = soup.find_all('table')
        df = pd.read_html(str(results_table))[0]
        all_tables.append(df)

    df = pd.concat(all_tables)

    return df

df = scrape_fbref_table_results(2017, 2023)

In [84]:
# Rename columns
df = df.rename(columns={
    'Sem.': 'Week',
    'Día': 'Day',
    'Fecha': 'Date',
    'Hora': 'Time',
    'Local': 'Home',
    'Marcador': 'Score',
    'xG.1': 'xg_away',
    'Visitante': 'Away',
    'Asistencia': 'Attendance',
    'Sedes': 'Venue'
})

# df.drop(['Árbitro', 'Informe del partido', 'Notas'], axis=1, inplace=True)

In [85]:
df.columns

Index(['Week', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xg_away', 'Away',
       'Attendance', 'Venue'],
      dtype='object')

In [86]:
# Display the first few rows of the dataframe
df.head()

,Week,Day,Date,Time,Home,xG,Score,xg_away,Away,Attendance,Venue
0,1.0,Vie,2017-08-18,20:15,Leganés,1.3,1–0,1.1,Alavés,9231.0,Estadio Municipal de Butarque
1,1.0,Vie,2017-08-18,22:15,Valencia,1.4,1–0,0.2,Las Palmas,35971.0,Estadio de Mestalla
2,1.0,Sáb,2017-08-19,18:15,Celta Vigo,1.8,2–3,2.1,Real Sociedad,16961.0,Estadio de Balaídos
3,1.0,Sáb,2017-08-19,20:15,Girona,2.2,2–2,0.7,Atlético Madrid,11511.0,Estadi Municipal de Montilivi
4,1.0,Sáb,2017-08-19,22:15,Sevilla,2.4,1–1,1.3,Espanyol,30487.0,Estadio Ramón Sánchez Pizjuán


In [87]:
df.isnull().sum()

Week          245
Day           245
Date          245
Time          245
Home          245
xG            245
Score         245
xg_away       245
Away          245
Attendance    733
Venue         245
dtype: int64

In [88]:
df = df.drop(['Attendance'], axis=1) # removing attendance since we don't have it for all games and won't know it for future games
df = df.dropna()

In [89]:
print(df.isnull().sum())

Week       0
Day        0
Date       0
Time       0
Home       0
xG         0
Score      0
xg_away    0
Away       0
Venue      0
dtype: int64


In [90]:
df = df.reset_index(drop=True)

In [91]:
# Convert 'Score' column into 'HomeGoals' and 'AwayGoals'
df['HomeGoals'] = df['Score'].str.split('–', expand=True)[0].astype(int)
df['AwayGoals'] = df['Score'].str.split('–', expand=True)[1].astype(int)

# Drop the original 'Score' column
df = df.drop(columns=['Score'])


In [92]:
# Calculate win rates for each team including draws
team_win_rates = {}
for team in pd.concat([df['Home'], df['Away']]).unique():
    home_wins = len(df[(df['Home'] == team) & (df['HomeGoals'] > df['AwayGoals'])])
    away_wins = len(df[(df['Away'] == team) & (df['AwayGoals'] > df['HomeGoals'])])
    draws = len(df[(df['Home'] == team) & (df['HomeGoals'] == df['AwayGoals'])]) + len(df[(df['Away'] == team) & (df['AwayGoals'] == df['HomeGoals'])])
    total_games = len(df[(df['Home'] == team) | (df['Away'] == team)])
    team_win_rates[team] = (home_wins + away_wins + 0.5 * draws) / total_games
    # This is something i contemplated for a bit but for the sake of a baseline model i left it at .5 for a draw

# Add win rate columns
df['HomeTeamWinRate'] = df['Home'].map(team_win_rates)
df['AwayTeamWinRate'] = df['Away'].map(team_win_rates)


In [80]:
team_win_rates['Barcelona']

0.7675438596491229

In [65]:
def match_result(row):
    if row['HomeGoals'] > row['AwayGoals']:
        return 'HomeWin'
    elif row['HomeGoals'] < row['AwayGoals']:
        return 'AwayWin'
    else:
        return 'Draw'

df['Result'] = df.apply(match_result, axis=1)

In [66]:
df.drop(['HomeGoals', 'AwayGoals'], axis=1, inplace=True)

In [72]:
# Convert 'Day' to numerical features indicating the day of the week
day_mapping = {
    'Lun': 0,  # Monday
    'Mar': 1,  # Tuesday
    'Mié': 2,  # Wednesday
    'Jue': 3,  # Thursday
    'Vie': 4,  # Friday
    'Sáb': 5,  # Saturday
    'Dom': 6,  # Sunday
}

df['Day'] = df['Day'].map(day_mapping)

# Convert 'Date' to year, month, and day features
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['DayOfMonth'] = pd.to_datetime(df['Date']).dt.day

# Assuming 'Time' is something like '15:00', '20:45', etc.
df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M').dt.hour

# Drop the original 'Date' and 'Time' columns
df = df.drop(columns=['Date', 'Time'])

In [73]:
df = df.drop(['Home', 'Away'], axis=1)

In [74]:
# One-hot encoding for 'Venue'
venue_dummies = pd.get_dummies(df['Venue'], prefix='Venue')

# Concatenate the original DataFrame and the dummy DataFrame
df = pd.concat([df, venue_dummies], axis=1)

# Drop the original 'Venue' column
df = df.drop(columns=['Venue'])

In [75]:
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = df.drop(columns=['Result'])
y = df['Result']

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [76]:
df.columns

Index(['Week', 'Day', 'xG', 'xg_away', 'HomeTeamWinRate', 'AwayTeamWinRate',
       'Result', 'Year', 'Month', 'DayOfMonth', 'Hour', 'Venue_Camp Nou',
       'Venue_Coliseum Alfonso Pérez', 'Venue_Estadi Municipal de Montilivi',
       'Venue_Estadio Abanca-Riazor', 'Venue_Estadio Alfredo Di Stéfano',
       'Venue_Estadio Benito Villamarín', 'Venue_Estadio Ciudad de Valencia',
       'Venue_Estadio Cívitas Metropolitano', 'Venue_Estadio El Alcoraz',
       'Venue_Estadio El Sadar', 'Venue_Estadio La Rosaleda',
       'Venue_Estadio Manuel Martínez Valero',
       'Venue_Estadio Municipal José Zorrilla',
       'Venue_Estadio Municipal de Anoeta',
       'Venue_Estadio Municipal de Butarque',
       'Venue_Estadio Municipal de Ipurúa',
       'Venue_Estadio Municipal de Riazor', 'Venue_Estadio Nuevo Los Cármenes',
       'Venue_Estadio Nuevo Mirandilla', 'Venue_Estadio Ramón Sánchez Pizjuán',
       'Venue_Estadio Ramón de Carranza', 'Venue_Estadio San Mamés',
       'Venue_Estadio San

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Create a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Predict the outcomes for the test set
y_pred = clf.predict(X_test)

# Print a classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     AwayWin       0.48      0.52      0.50       121
        Draw       0.36      0.16      0.22       122
     HomeWin       0.60      0.77      0.68       213

    accuracy                           0.54       456
   macro avg       0.48      0.48      0.46       456
weighted avg       0.51      0.54      0.51       456



### Thoughts on the results

Obviously there are a lot of improvements we can make to this model.

For example:
- We can try different models and hyperparameters
- We can try different features
- We can try different ways of encoding the features
- We can try different ways of handling missing data
- We can try different ways of handling draws
- etc.

The point of this notebook wasn't necessarily to show how to build the best possible model, but rather show that GPT-4/ChatGPT can be used to improve workflows and make data science more accessible to people who don't have a background in data science.

In the space of about 60 minutes I was able to go from no model to a baseline model I can start improving and iterating on.
